In [40]:
# new virtual env every time?
# !pip install --upgrade pip
# !pip install nltk==3.5
# TODO: doesnt work?? !pip install numpy==1.19.5
# !pip install sklearn==0.0  
# !pip install torch #==1.7.1   
# !pip install transformers==3.0.2   
# !pip install --upgrade OpenHowNet


  Attempting uninstall: OpenHowNet
    Found existing installation: OpenHowNet 0.0.1a11
    Uninstalling OpenHowNet-0.0.1a11:
      Successfully uninstalled OpenHowNet-0.0.1a11


In [1]:
import torch
from transformers import BertTokenizer
import numpy as np
import OpenHowNet
import nltk


In [2]:
# load english dataset
def load_data(path):
    # return words list and labels
    with open(path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        lines = [line.strip().lower().split('\t') for line in lines]
        train_former = [line[0] for line in lines[:101171]]
        train_quote = [line[1] for line in lines[:101171]]
        train_latter = [line[2] for line in lines[:101171]]
        valid_former = [line[0] for line in lines[101171:113942]]
        valid_quote = [line[1] for line in lines[101171:113942]]
        valid_latter = [line[2] for line in lines[101171:113942]]
        test_former = [line[0] for line in lines[113942:]]
        test_quote = [line[1] for line in lines[113942:]]
        test_latter = [line[2] for line in lines[113942:]]
        all_quotes = train_quote + valid_quote + test_quote
    all_quotes = list(set(all_quotes))
    all_quotes.sort()
    y_train = [all_quotes.index(q) for q in train_quote]
    y_valid = [all_quotes.index(q) for q in valid_quote]
    y_test = [all_quotes.index(q) for q in test_quote]

    return train_former, train_latter, train_quote, valid_former, valid_latter, valid_quote, test_former, test_latter, test_quote, torch.LongTensor(
        y_train), torch.LongTensor(y_valid), torch.LongTensor(
            y_test), all_quotes

In [3]:

print("loading dataset......")
data_path = "./data/english.txt"
train_former, train_latter, train_quote, valid_former, valid_latter, valid_quote, test_former, test_latter, test_quote, y_train, y_valid, y_test, all_quotes = load_data(
    data_path)
print("tran  valid  test:", len(train_former), len(valid_former), len(test_former))
print("all quotes: ", len(all_quotes))
print("train quote:", len(list(set(train_quote))))
print("valid quote:", len(list(set(valid_quote))))
print("test quote:", len(list(set(test_quote))))

loading dataset......
tran  valid  test: 101171 12771 12771
all quotes:  6108
train quote: 6008
valid quote: 6108
test quote: 6108


In [7]:
# load pretrained model
PRETRAINED_MODEL_NAME = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)


all_quote_words = []
for s in all_quotes:
    q_words = [w for w in nltk.word_tokenize(s)]
    all_quote_words.extend(q_words)
all_quote_words = list(set(all_quote_words))
all_quote_words.sort()
all_quote_words.insert(0, '<INS>')
all_quote_words.insert(101, '<INS>')
all_quote_words.insert(102, '<INS>')
print("all quote words ", len(all_quote_words))
print(all_quote_words[0])
print(all_quote_words[101])
print(all_quote_words[102])

Important data file "resources/sememe_all" lost, please run `OpenHowNet.download()`.
all quote words  7636
<INS>
<INS>
<INS>


In [27]:
# I notice that many of the quote words seem to have apostrophe in front of them
# Seems like hownet isn't able to deal w that
nltk.word_tokenize(all_quotes[0])

["'nothing", 'tastes', 'as', 'good', 'as', 'skinny', 'feels', '.', "'"]

In [8]:
# get bert-base-uncased vocab
vocab = tokenizer.vocab
print("vocab size: ", len(vocab))
all_tokens = [token for token, idx in vocab.items()]
print("all vocab token: ", len(all_tokens))

vocab size:  30522
all vocab token:  30522


In [11]:
# OpenHowNet.download()
hownet_dict = OpenHowNet.HowNetDict()

Initializing OpenHowNet succeeded!


In [48]:
# get all english sememes
all_semems = []
for i in range(len(all_quote_words)):
    # semems = hownet_dict.get_sememes_by_word(all_quote_words[i], structured=False, lang="en", merge=True)
    semems = hownet_dict.get_sememes_by_word(all_quote_words[i], display='list', merge=True)
    for s in semems:
        all_semems.append(s)
all_semems = list(set(all_semems))
all_semems.sort()
print("all sememes: ", len(all_semems))

TypeError: '<' not supported between instances of 'Sememe' and 'Sememe'

In [1]:
all_quote_words[0]

NameError: name 'all_quote_words' is not defined

In [52]:
all_semems[0]

BeBeyond|越界

In [54]:
# Generate sememes for each word
sememe_onehot = np.zeros((len(all_quote_words), len(all_semems)))
for i in range(len(all_quote_words)):
    semems = hownet_dict.get_sememes_by_word(all_quote_words[i], display='list', merge=True)
    print(semems)
    if len(semems) > 0:
        word2sememe = [s for s in semems]
        for s in word2sememe:
            sememe_onehot[i][all_semems.index(s)] = 1
print("word2sememe one hot: ", sememe_onehot.shape)
np.save("./data/english_word_sememe.npy", sememe_onehot)

[]
[punc|标点]
[money|货币, mean|指代, symbol|符号, commerce|商业, US|美国, finance|金融, Unit|单位]
[math|数学, Rate|比率, cardinal|基数]
[punc|标点]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[FuncWord|功能词]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[punc|标点]
[subtract|削减, punc|标点, DoSum|算术, symbol|符号]
[punc|标点]
[cardinal|基数]
[cardinal|基数]
[]
[cardinal|基数]
[cardinal|基数]
[]
[]
[]
[cardinal|基数]
[cardinal|基数]
[]
[cardinal|基数]
[cardinal|基数]
[cardinal|基数]
[cardinal|基数]
[cardinal|基数]
[]
[cardinal|基数]
[cardinal|基数]
[punc|标点]
[punc|标点]
[punc|标点]
[NounUnit|名量, UK|英国, character|文字, cardinal|基数, economy|经济, InstitutePlace|场所, medical|医, physical|物质, FuncWord|功能词]
[]
[]
[]
[facilities|设施, religion|宗教]
[]
[hate|仇恨, disgust|厌恶]
[]
[]
[]
[]
[]
[TimeLong|长时间]
[]
[animate|生物, human|人, Ability|能力]
[able|能, Vable|有能力, FuncWord|功能词]
[house|房屋, reside|住下, FuncWord|功能词]
[remove|消除]
[]
[]
[disgust|厌恶]
[labour|临产, fail|失败]
[medical|医, cease|停做, fail|失败, labour|临产, pregnant|怀孕]
[exist|存在, many|多]
[]
[roughly|大概, FuncWord|功能词]
[

In [55]:
# Generate the word index for each quote
all_word_ids = []
all_input_ids = []
for quote in all_quotes:
    quote_words = [w for w in nltk.word_tokenize(quote)]
    token2word = []
    for w in quote_words:
        tokens = tokenizer.tokenize(w)
        ids = tokenizer.convert_tokens_to_ids(tokens)
        for id in ids:
            token2word.append([id, all_quote_words.index(w)])

    encoded_dict = tokenizer.encode_plus(quote,
                                         add_special_tokens=True,
                                         max_length=80,
                                         pad_to_max_length=True,
                                         truncation=True,
                                         return_attention_mask=True,
                                         return_tensors='pt')
    input_ids = encoded_dict['input_ids']
    all_input_ids.append(input_ids)
    input_ids = input_ids.squeeze().tolist()
    word_ids = []
    for id in input_ids:
        if id == 101:
            word_ids.append(id)
        elif id == 102:
            word_ids.append(id)
        elif id == 0:
            word_ids.append(id)
        else:
            for i in range(len(token2word)):
                if id == token2word[i][0]:
                    word_ids.append(token2word[i][1])
                    token2word = token2word[i+1:]
                    break
    if len(word_ids) != 80:
        word_ids.extend(0 for _ in range(80-len(word_ids)))
    all_word_ids.append(torch.LongTensor(word_ids))

all_word_ids = torch.stack(all_word_ids, dim=0)
print("all word ids: ", all_word_ids.shape)
all_input_ids = torch.cat(all_input_ids, dim=0)
print("all input ids: ", all_input_ids.shape)

/Users/abhi/anaconda3/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


all word ids:  torch.Size([6108, 80])
all input ids:  torch.Size([6108, 80])


In [56]:
torch.save(all_word_ids, "./data/bert_base_uncased_quote2word.pt")